# 1. Identify the data source, requirements

Import libraries

In [ ]:
import pandas as pd

# library helper
# run: importnb-install from Conda before using
from importnb import Notebook
with Notebook(): 
    import Utility

# custom helper class (from jupyter notebook)
helper = Utility.Helper()

from bs4 import BeautifulSoup
import requests

import re

from string import ascii_lowercase

# progress bars for long running functions
from tqdm import tqdm

In [ ]:
# reload changes in Jupyter notebooks
from importlib import reload
with Notebook(): __name__ == '__main__' and reload(Utility)

In [ ]:
DATA_PATH = '../../data/'

SITE_URL = 'https://www.lyrics.com/'

# 2. Identify data format, establish connection, collect sample data

In [ ]:
genres_url = SITE_URL + "all-genres/"
genres_html = requests.get(genres_url).text
genres_soup = BeautifulSoup(genres_html, 'html5lib')

genres_container = genres_soup.find('div', id='content-body').find('div', class_= re.compile("tags-list")).find_all('div')

In [ ]:
genre_list = []
genre_url = []

for genre_container in genres_container:
    genre_list.append(genre_container.find('a').text)
    genre_url.append(genre_container.find('a').get('href'))

# add pages per genre: 
# ['Blues','Brass & Military','Children','Classical', 'Electronic','Folk, World, & Country',
# 'Funk / Soul','Hip Hop','Jazz', 'Latin','Non-Music','Pop','Reggae','Rock','Spiritual','Stage & Screen' ]
genre_pages = [217, 2, 0, 34, 881, 765, 580, 596, 668, 241, 15, 1887, 134, 2668, 0, 119]

genre_data = {'Genre Name': genre_list, 'Genre URL': genre_url, 'Genre Pages': genre_pages}

genre_df = pd.DataFrame(data=genre_data)

In [ ]:
gname = []
sname = []
surl = []

for i in tqdm(range(0, len(genre_list))):
    
    url = SITE_URL + genre_url[i]
    url = url.replace(" ", "%20")
    
    # loop through pages
    for j in tqdm(range(1, genre_pages[i])):
        
        # base case, append page #
        j_url = url + "/" + str(j)
        
        if (j == 1):
            # 1st page doesn't need page #
            j_url = url
            
        genre_html = requests.get(j_url).text
        genre_soup = BeautifulSoup(genre_html, 'html5lib')
        songs_soup = genre_soup.find('div', id='content-body').find_all('div', class_= re.compile('lyric-meta'))

        for song_soup in songs_soup:

            gname.append(genre_list[i])
            sname.append(song_soup.find('p', class_= re.compile('lyric-meta')).find('a').text)
            surl.append(song_soup.find('p', class_= re.compile('lyric-meta')).find('a').get('href'))

In [ ]:
d = {'Song Title': sname, 'Song URL': surl, 'Genre': gname}

songsByGenre = pd.DataFrame(data=d)

In [ ]:
print("Number of songs by genre:", songsByGenre.shape[0])

songsByGenre.head()

In [ ]:
songsByGenre.to_csv(DATA_PATH + "songsByGenrePages.csv", index=False)